# Conversational Interface - Chatbot with Claude-v2 and OpenSearch

> 이 노트북은 SageMaker Studio의 **`Data Science 3.0`** 커널과 잘 작동합니다.

> **SageMaker Notebook Instance** 를 이용해 실습을 진행하신다면, **JupyterLab이 아닌 Jupyter**에서 실행하시기 바랍니다.

이 노트북에서는 Amazon Bedrock의 기본 모델 (FM) 을 사용하여 챗봇을 구축할 것입니다.사용 사례에서는 Claude를 챗봇 구축을 위한 FM으로 사용합니다.

---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

## 개요

챗봇 및 가상 어시스턴트와 같은 대화형 인터페이스를 사용하여 고객의 사용자 경험을 향상시킬 수 있습니다. 챗봇은 자연어 처리 (NLP) 및 기계 학습 알고리즘을 사용하여 사용자 쿼리를 이해하고 이에 응답합니다.챗봇은 고객 서비스, 판매, 전자 상거래와 같은 다양한 애플리케이션에서 사용되어 사용자에게 빠르고 효율적인 응답을 제공할 수 있습니다.웹사이트, 소셜 미디어 플랫폼 및 메시징 앱과 같은 다양한 채널을 통해 액세스할 수 있습니다.


## Amazon Bedrock을 사용하는 챗봇


### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.


이 노트북에는 몇 가지 추가 종속성도 필요합니다.

- [iPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), 노트북의 대화형 UI 위젯용


In [2]:
# install_needed = True  # should only be True once
install_needed = False  # should only be True once

In [3]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker    
    !{sys.executable} -m pip install "ipywidgets>=7,<8"
    !{sys.executable} -m pip install langchain==0.0.279    

    
    IPython.Application.instance().kernel.do_shutdown(True)

In [4]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [5]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [6]:
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm_text = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, 
              model_kwargs={
                              'max_tokens_to_sample':1024,
                              'temperature': 0.1,
                              'top_k': 3,       
                              'top_p': 0.1,                                                                   
                           })

llm_text

Bedrock(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<botocore.client.Bedrock object at 0x7f4c7c20c3d0>, region_name=None, credentials_profile_name=None, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 1024, 'temperature': 0.1, 'top_k': 3, 'top_p': 0.1}, endpoint_url=None)

## Embedding 모델 선택

In [7]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(client=boto3_bedrock)
        print("Bedrock Embeddings Model Loaded")
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [8]:
# is_bedrock_embeddings = True
is_bedrock_embeddings = False
is_KoSimCSERobert = True
#is_KoSimCSERobert = False

##############################
# Parameters for is_KoSimCSERobert
##############################
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
endpont_name = "KoSimCSE-roberta-2023-09-01-14-34-11"
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

KoSimCSERobert Embeddings Model Loaded


# 3. LangChain OpenSearch VectorStore 생성 
## 선수 조건
- 이전 노트북 02_KR_RAG_OpenSearch_Claude.ipynb 를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.

## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 OpenSearch ID/PW 를 입력을 해주세요.

In [9]:

os.environ["OpenSearch_UserName"] = "Type ID" 
os.environ["OpenSearch_UserPassword"] = "Type PW" 

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]


In [10]:
# OpenSearch Dashboards URL = https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com/_dashboards
opensearch_domain_endpoint = "https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password

index_name = "fsi-v10"


## LangChain OpenSearch VectorStore 생성

In [11]:
from langchain.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

# 4. RetrievalQA 구현

## 템플릿 생성

In [12]:
from utils.rag import create_bool_filter, run_RetrievalQA, show_context_used
from langchain.prompts import PromptTemplate

prompt_template = """Human: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 읃답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


### 필터 생성

In [13]:
filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

# query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
query = "타기관OTP 등록 방법 알려주세요"

boolean_filter = create_bool_filter(filter01, filter02)


In [14]:
result = run_RetrievalQA(query, boolean_filter, llm_text, vector_db, PROMPT, verbose=False, is_filter=True ,k=5)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])
# show_context_used(result['source_documents'])        


##################################
query:  타기관OTP 등록 방법 알려주세요
##################################
 타기관 OTP 등록 방법은 다음과 같습니다:

1. 인터넷뱅킹 로그인
2. 사용자관리 > 인터넷뱅킹관리 > OTP이용등록
3. 신한 쏠(SOL) 로그인
4. 전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록

※ OTP이용등록 후 재로그인하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금한 점은 신한은행 고객센터(1599-8000)로 문의하시기 바랍니다.


# 5. Conversational Chatbot 사용한 대화형 세션

다음 유틸리티 클래스를 사용하면 Claude와 보다 자연스러운 방식으로 상호 작용할 수 있습니다. 입력창에 질문을 적고 클로드의 답변을 받습니다. 그러면 대화를 계속할 수 있습니다.

### 메모리
모든 챗봇에는 사용 사례에 따라 맞춤화된 다양한 옵션을 갖춘 QA 체인이 필요합니다. 그러나 챗봇에서는 모델이 답변을 제공하기 위해 이를 고려할 수 있도록 항상 대화 기록을 보관해야 합니다. 이 예에서는 대화 기록을 유지하기 위해 ConversationBufferMemory와 함께 LangChain의 [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db)을 사용합니다.

출처: https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db

뒤에서 무슨 일이 일어나고 있는지 모두 보려면 'verbose'를 'True'로 설정하세요.

### ConversationRetrievalChain에 사용되는 매개변수
* **retriever**: 우리는 `VectorStore`가 지원하는 `VectorStoreRetriever`를 사용했습니다. 텍스트를 검색하려면 `"similarity"` 또는 `"mmr"`라는 두 가지 검색 유형을 선택할 수 있습니다. `search_type="similarity"`는 질문 벡터와 가장 유사한 텍스트 청크 벡터를 선택하는 검색 객체에서 유사성 검색을 사용합니다.

* **메모리**: 이력을 저장하는 메모리 체인


질문이 컨텍스트 범위를 벗어나면 모델은 답을 모른다고 대답합니다.

**참고**: 체인이 어떻게 작동하는지 궁금하다면 `verbose=True` 줄의 주석 처리를 해제하세요.

## ConversationChain 생성

In [15]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage
# from utils.rag import _get_chat_history 


from langchain import PromptTemplate

In [16]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

general_system_template = r""" 
Use the following pieces of context to answer the question at the end based on the Information not using ask. If you don't know the answer, just say that you don't know, don't try to make up an answer.
 ----
{context}
----
"""

general_user_template = "{question}"
messages = [
            SystemMessagePromptTemplate.from_template(general_system_template),
            HumanMessagePromptTemplate.from_template(general_user_template)
]
qa_prompt = ChatPromptTemplate.from_messages( messages )

# print(qa_prompt)

### 챗봇 (ConversationalRetrievalChain)  질문

In [17]:
from utils.rag import ChatUX   

def create_ConversationalRetrievalChain(qa_prompt, boolean_filter, llm_text, k=1, verbose=False):
    memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    qa = ConversationalRetrievalChain.from_llm(
        llm= llm_text, 
        retriever=vector_db.as_retriever(
                            search_type="similarity", 
                            search_kwargs={
                                            "k": k,
                                            "boolean_filter" : boolean_filter
                                        }
                ),
        memory=memory_chain,
        combine_docs_chain_kwargs={'prompt': qa_prompt},                                               
    #    get_chat_history=_get_chat_history,
    #     verbose=True,
        verbose= verbose,    
        # condense_question_prompt=condense_prompt_claude, 
        # chain_type='stuff', # 'refine',
    )
    
    return qa
    


# 6. 챗봇 (ConversationalRetrievalChain) 실행

## (1) 챗봇 실행

In [18]:
qa = create_ConversationalRetrievalChain(qa_prompt, boolean_filter, llm_text, k=3, verbose=False)

In [19]:
qa("타기관OTP 등록 방법 알려주세요")

{'question': '타기관OTP 등록 방법 알려주세요',
 'chat_history': [HumanMessage(content='타기관OTP 등록 방법 알려주세요', additional_kwargs={}, example=False),
  AIMessage(content=".\n\nAssistant: 네, 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하여 이용이 가능합니다.\n\n인터넷뱅킹의 경우 로그인 후 '사용자관리 > 인터넷뱅킹관리 > OTP이용등록' 메뉴에서 등록이 가능합니다. \n\n모바일뱅킹 신한쏠의 경우 로그인 후 '전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록'에서 등록이 가능합니다.\n\n등록 후에는 재로그인을 해주셔야 새로 등록한 보안매체가 적용됩니다.", additional_kwargs={}, example=False)],
 'answer': ".\n\nAssistant: 네, 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하여 이용이 가능합니다.\n\n인터넷뱅킹의 경우 로그인 후 '사용자관리 > 인터넷뱅킹관리 > OTP이용등록' 메뉴에서 등록이 가능합니다. \n\n모바일뱅킹 신한쏠의 경우 로그인 후 '전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록'에서 등록이 가능합니다.\n\n등록 후에는 재로그인을 해주셔야 새로 등록한 보안매체가 적용됩니다."}

In [20]:
qa("등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?")

{'question': '등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?',
 'chat_history': [HumanMessage(content='타기관OTP 등록 방법 알려주세요', additional_kwargs={}, example=False),
  AIMessage(content=".\n\nAssistant: 네, 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하여 이용이 가능합니다.\n\n인터넷뱅킹의 경우 로그인 후 '사용자관리 > 인터넷뱅킹관리 > OTP이용등록' 메뉴에서 등록이 가능합니다. \n\n모바일뱅킹 신한쏠의 경우 로그인 후 '전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록'에서 등록이 가능합니다.\n\n등록 후에는 재로그인을 해주셔야 새로 등록한 보안매체가 적용됩니다.", additional_kwargs={}, example=False),
  HumanMessage(content='등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?', additional_kwargs={}, example=False),
  AIMessage(content='\n\nAssistant: 네, 맞습니다. 타기관에서 발급받은 OTP를 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하신 후 재로그인하시면 새로 등록한 OTP를 사용하실 수 있습니다. OTP 등록 후 반드시 재로그인을 하셔야 새로 등록한 보안매체가 적용됩니다.', additional_kwargs={}, example=False)],
 'answer': '\n\nAssistant: 네, 맞습니다. 타기관에서 발급받은 OTP를 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하신 후 재로그인하시면 새로 등록한 OTP를 사용하실 수 있습니다. OTP 등록 후 반드시 재로그인을 하셔야 새로 등록한 보안매체가 적용됩니다.'}

## (2) 챗봇 실행 (내부 로그 확인)

In [21]:
qa_d = create_ConversationalRetrievalChain(qa_prompt, boolean_filter, llm_text, k=3, verbose=True)

In [22]:
qa_d("타기관OTP 등록 방법 알려주세요")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  
Use the following pieces of context to answer the question at the end based on the Information not using ask. If you don't know the answer, just say that you don't know, don't try to make up an answer.
 ----
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 공과금 자동이체 신청이 가능한가요?
Information: 인터넷뱅킹에 로그인하신 후 "공과금/법원 > 공과금센터" 페이지에 가시면 "지로자동이체 등록" 메뉴가 있습니다. 해당 메뉴를 통하여 "전기요금, 전화요금, 국민연금, 국민건강보험료 등을 포함하여 각종 지로요금"을 모두 자동이체 등록하실 수 있습니다.

ask: 모바일 OTP 사용하고 있는데 인터넷뱅킹에서는 어떻게 이용하는건가요?
Information: 인터넷뱅킹 거래시 보안매체 입력단계에 [인증요청] 버튼 클릭하면 핸드폰으로 푸시메세지가 전달됩니다. 
푸시메세지 터치하여 모바일 OTP 비밀번호 6자리 입력, 인증처리 완료후 인터넷뱅킹에서 이후의 절차를 진행하시면 됩니다.
※ 스마트폰에 모바일 OTP 푸시(PUSH) 메세지가 오지 않는 경우 메뉴

{'question': '타기관OTP 등록 방법 알려주세요',
 'chat_history': [HumanMessage(content='타기관OTP 등록 방법 알려주세요', additional_kwargs={}, example=False),
  AIMessage(content=".\n\nAssistant: 네, 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하여 이용이 가능합니다.\n\n인터넷뱅킹의 경우 로그인 후 '사용자관리 > 인터넷뱅킹관리 > OTP이용등록' 메뉴에서 등록이 가능합니다. \n\n모바일뱅킹 신한쏠의 경우 로그인 후 '전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록'에서 등록이 가능합니다.\n\n등록 후에는 재로그인을 해주셔야 새로 등록한 보안매체가 적용됩니다.", additional_kwargs={}, example=False)],
 'answer': ".\n\nAssistant: 네, 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하여 이용이 가능합니다.\n\n인터넷뱅킹의 경우 로그인 후 '사용자관리 > 인터넷뱅킹관리 > OTP이용등록' 메뉴에서 등록이 가능합니다. \n\n모바일뱅킹 신한쏠의 경우 로그인 후 '전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록'에서 등록이 가능합니다.\n\n등록 후에는 재로그인을 해주셔야 새로 등록한 보안매체가 적용됩니다."}

In [23]:
qa_d("등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 타기관OTP 등록 방법 알려주세요
Assistant: .

Assistant: 네, 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하여 이용이 가능합니다.

인터넷뱅킹의 경우 로그인 후 '사용자관리 > 인터넷뱅킹관리 > OTP이용등록' 메뉴에서 등록이 가능합니다. 

모바일뱅킹 신한쏠의 경우 로그인 후 '전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록'에서 등록이 가능합니다.

등록 후에는 재로그인을 해주셔야 새로 등록한 보안매체가 적용됩니다.
Follow Up Input: 등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  
Use the following pieces of context to answer the question at the end based on the Information not using ask. If you don't know the answer, just say that you don't know, don't try to make up an answer.
 ----
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에

{'question': '등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?',
 'chat_history': [HumanMessage(content='타기관OTP 등록 방법 알려주세요', additional_kwargs={}, example=False),
  AIMessage(content=".\n\nAssistant: 네, 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하여 이용이 가능합니다.\n\n인터넷뱅킹의 경우 로그인 후 '사용자관리 > 인터넷뱅킹관리 > OTP이용등록' 메뉴에서 등록이 가능합니다. \n\n모바일뱅킹 신한쏠의 경우 로그인 후 '전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록'에서 등록이 가능합니다.\n\n등록 후에는 재로그인을 해주셔야 새로 등록한 보안매체가 적용됩니다.", additional_kwargs={}, example=False),
  HumanMessage(content='등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?', additional_kwargs={}, example=False),
  AIMessage(content='\n\nAssistant: 네, 맞습니다. 타기관에서 발급받은 OTP를 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하신 후 재로그인하시면 새로 등록한 OTP를 사용하실 수 있습니다. OTP 등록 후 반드시 재로그인을 하셔야 새로 등록한 보안매체가 적용됩니다.', additional_kwargs={}, example=False)],
 'answer': '\n\nAssistant: 네, 맞습니다. 타기관에서 발급받은 OTP를 신한은행 인터넷뱅킹이나 모바일뱅킹에 등록하신 후 재로그인하시면 새로 등록한 OTP를 사용하실 수 있습니다. OTP 등록 후 반드시 재로그인을 하셔야 새로 등록한 보안매체가 적용됩니다.'}

## ipywidgets를 대화형 챗봇

다음과 같은 질문을 넣어 보세요

- 타기관OTP 등록 방법 알려주세요
- 등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?


In [24]:
qa_1 = create_ConversationalRetrievalChain(qa_prompt, boolean_filter, llm_text, k=3, verbose=False)

chat = ChatUX(qa_1, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

# Reference
Chat Over Documents with Vectara
- https://blog.langchain.dev/conversational-retrieval-agents/